In [ ]:
def get_disruption_value(focal_repository):
    
    
    # Get the data for the the focal repository only
    query='SELECT * FROM gitdatam WHERE repo_name = %s'
    df = pd.read_sql(query, con=db, params=[focal_repository])
    # pass the dataframe into the fuction containing all the types pertaining to one repository
    def contributors_fun(df):
        Contributors=[]
        Contributors=df.loc[(df['type']=="PullRequestEvent")&(df['payload_pull_request_merged']=="TRUE")].actor_login.tolist()
        unique_contributors= set(Contributors)
        return unique_contributors


    #calling function to get the contributors to a repository
    contributors=contributors_fun(df)


    #print contributors
    #print("The contributors for the focal repository are {}".format(contributors))
    print("The focal repository is {}".format(focal_repository))
    print("The number of contributors are {0}".format(len(contributors)))
    if len(contributors)==0:
        D="NaN"
        Parameters=[D,len(contributors)]
        return Parameters
    else:


        #function to find core-contributors
        #pass the dataframe consisting only the records of repository for which you want to find the core contributors
        #useful function if you have a single repository and a list of unique contributors
        def core_contributors_fun(unique_contributors,Main_df):
            #Showing the push events by all the contributors to dynamo-easy repository by putting them in a dataframe
            Contributors_df=pd.DataFrame()
            Contributors_df=Main_df.loc[(df['type']=="PushEvent")&(Main_df.actor_login.isin(unique_contributors))].copy()
            # forming a dictionary of count of each contributor as the value and contributor as key.
            from collections import defaultdict
            contributors_push_count = defaultdict(list)
            temp_list=[]
            for key in unique_contributors:
                temp_list.append(key)
                contributors_push_count[key]=Contributors_df.actor_login.isin(temp_list).sum()
                temp_list.pop()

            #finding out the number of pushes of all the contributors for dynamo-easy repository
            Total_pushes_count=Contributors_df.id.count()

            #Average of all the pushes by the contributors
            Average_pushes=Total_pushes_count/len(unique_contributors)

            #Compare the average number of pushes of all the contributors to number of pushes to individual contributors in 
            #unique contributors list
            Core_contributors=[]
            for key,value in contributors_push_count.items():
                if value > Average_pushes:
                    Core_contributors.append(key)
            return Core_contributors


    #calling the function to get core-contributors
        core_contributors_main=core_contributors_fun(contributors,df)
        #print("The core-contributors to the focal repository are {}".format(core_contributors_main))
        print("The number of core contributors are {0}".format(len(core_contributors_main)))


        #function to get the references for the current repositories
        #pass the core contributors and the data frame consisting all types of entries.
        def reference_repos(Core_contributors,Main_df,core_df_all):

            # making a data frame which has only the core-contributors
            core_df=Main_df[Main_df.actor_login.isin(Core_contributors)]
            time_df = pd.DataFrame(columns=['core_contributors','start_date','end_date'])
            #core_df.groupby('actor_id').groups
            grouped = core_df.groupby('actor_login')
            for name,group in grouped:
                dict_row={'core_contributors':name,'start_date':group.created_at.min(),'end_date':group.created_at.max()}
                time_df=time_df.append(dict_row,ignore_index=True)
            Final_Repositories=[]

            for index,row in time_df.iterrows():
                row_start_date=row['start_date']
                row_end_date=row['end_date']
                Final_Repositories.extend(core_df_all.loc[(core_df_all['actor_login']==row['core_contributors'])&(core_df_all.created_at.between(row_start_date,row_end_date))&(core_df_all.type=="ForkEvent")].repo_name)
            return list(set(Final_Repositories))



        #Getting the dataframe consisting of all repositories related to the core-contributors
        if len(core_contributors_main)==0:
            D="NaN"
            Parameters=[D,len(contributors),len(core_contributors_main)]
            return Parameters
        else:
            if len(core_contributors_main)==1:
                query = 'SELECT * FROM gitdatam WHERE actor_login = %s '
                core_df_all = pd.read_sql(query, con=db,params=[core_contributors_main[0]])

            else:
                core_contributors_t = tuple(core_contributors_main)
                query = "SELECT * FROM gitdatam WHERE actor_login IN {}".format(core_contributors_t)
                core_df_all = pd.read_sql(query, con=db)
            core_df_all.head()


            #calling the function to get the reference repositories
            reference_repos_list=reference_repos(core_contributors_main,df,core_df_all)
            if len(reference_repos_list)==0:
                D="NaN"
                Parameters=[D,len(contributors),len(core_contributors_main)]
                return Parameters
    

            #printing reference repos
            #print("The reference repos for the focal repository are : {}".format(reference_repos_list))



            #calling function to get the contributors to the focal repository

            #print("The contributors to the focal repository are:{}".format(contributors))


            #Getting the dataframe consisting of all repositories related to the contributors
            if len(contributors)==1:
                query = "SELECT * FROM gitdatam WHERE actor_login = %s "
                con_df = pd.read_sql(query, con=db,params=[contributors[0]])

            else:
                contributors_t = tuple(contributors)
                query = "SELECT * FROM gitdatam WHERE actor_login IN {}".format(contributors_t)
                con_df = pd.read_sql(query, con=db)


            con_df.head()

            # Let df contain all the data which has only contributors related data
            #find all the repos for which these actor_logins(contributors for current repository) have made a pull request and it is merged
            from collections import defaultdict
            contributors_repos= defaultdict(list)
            repo_list=[]
            for actor in contributors:
                repos=[]
                repos=con_df.loc[(con_df['type']=="PullRequestEvent")&(con_df['payload_pull_request_merged']=="TRUE")&(con_df['actor_login']==actor)].repo_name.tolist()
                repo_list.extend(repos)
                #import pdb; pdb.set_trace()
                repos=list(set(repos))
                repos.remove(focal_repository)
                contributors_repos[actor]=repos
            repo_list=list(set(repo_list))

            #for key,value in contributors_repos.items():
               # print(key,value)



            # Getting the number of repositories where each contributor  of focal rep is also a contributor to other repos
            # for key,value in contributors_repos.items():
            #     print(key,len(value))

            # function to return if the given single_contributor is a core_contributor or no
            def core_contributors_bool_fun(unique_contributors, single_contributor, Main_df):
                # Showing the push events by all the contributors to dynamo-easy repository by putting them in a dataframe
                Contributors_df = pd.DataFrame()
                Contributors_df = Main_df.loc[(df['type'] == "PushEvent") & (Main_df.actor_login.isin(unique_contributors))].copy()

                # Finding the number of counts made by the single contributor
                single_contributors_push_count = Contributors_df.loc[Contributors_df['actor_login'] == single_contributor].id.count()

                # finding out the number of pushes of all the contributors for dynamo-easy repositor
                Total_pushes_count = Contributors_df.id.count()

                # Average of all the pushes by the contributors
                Average_pushes = Total_pushes_count / len(unique_contributors)

                # Compare the average number of pushes of all the contributors to number of pushes to individual contributors in
                # unique contributors list


                if single_contributors_push_count > Average_pushes:
                        decider = True
                else:
                        decider = False

                return decider


            #Getting all the repos for contributed by the contributors
            if len(repo_list)==0:
                D="NaN"
                Parameters=[D,len(contributors),len(core_contributors_main)]
                return Parameters
            if len(repo_list)==1:
                query = "SELECT * FROM gitdatam WHERE repo_name = %s "
                repo_all = pd.read_sql(query, con=db,params=[repo_list[0]])

            else:
                repo_t = tuple(repo_list)
                query = "SELECT * FROM gitdatam WHERE repo_name IN {}".format(repo_t)
                repo_all = pd.read_sql(query, con=db)

            repo_all.head()


            # Creating a dictionary of core_contributor_repos consisting of core contributor as key and the list of their
            # repositories as values
            def core_contributors_repo_dict_fun(contributors_repos,repo_all):
                from collections import defaultdict
                #import pdb; pdb.set_trace()
                core_contributors_repos = defaultdict(list)
                core_repos = []
                unique_contributors_temp = []

                for key, value in contributors_repos.items():
                    for repo in value:
                        # Filtering only the entries where the repo_name is repo for which type is push, fork or pull request and storing them in a new dataframe
                        repo_df = repo_all.loc[repo_all['repo_name'] == repo]
                        unique_contributors_temp = contributors_fun(repo_df)
                        if unique_contributors_temp != []:
                            decider = core_contributors_bool_fun(unique_contributors_temp, key, repo_df)
                            if decider == True:
                                core_repos.append(repo)
                    if core_repos != []:
                        core_repos = list(set(core_repos))

                        core_contributors_repos[key] = core_repos
                    core_repos = []

                return core_contributors_repos



            # calling the function core_contributors_repo_dict_fun function
            core_contributors_repos=core_contributors_repo_dict_fun(contributors_repos,repo_all)
            #print(core_contributors_repos)


            #making a list of all the keys(cc's in core_contributors_repos)
            cc_list_other=[]
            for key,value in core_contributors_repos.items():
                cc_list_other.append(key)
            #print(cc_list_other)

            #getting the all the data related to the core contributors in the core_contributors_repos into the data frame
            if len(cc_list_other)==0:
                D="NaN"
                Parameters=[D,len(contributors),len(core_contributors_main)]
                return Parameters
            if len(cc_list_other)==1:
                query = "SELECT * FROM gitdatam WHERE actor_login = %s "
                cc_df_other = pd.read_sql(query, con=db,params=[cc_list_other[0]])

            else:
                cc_list_other_t = tuple(cc_list_other)
                query = "SELECT * FROM gitdatam WHERE actor_login IN {}".format(cc_list_other_t)
                cc_df_other = pd.read_sql(query, con=db)


            cc_df_other.head()


            # function to get decide if a cc has forked current repository in the active period of a repo
            #pass the cc, focal repository(current_repo), main_repo(repo for which the cc has worked during the active period)
            def reference_repos_bool(Core_contributor,current_repo, main_repo, Main_df):
                # making a data frame which has only the entries of the core_contributor and the main_repo
                core_df=Main_df.loc[(Main_df.actor_login == Core_contributor)&(Main_df.repo_name == main_repo) & (Main_df.type== "PushEvent")].copy()
                # Getting the  date of the first push  on main_repo
                start_date=core_df.created_at.min()
                # Getting the date of the last push on main_repo
                end_date=core_df.created_at.max()
                # Getting the count of all the fork events on current_repo by the core_contributor
                Fork_count_current_repo= Main_df.loc[(Main_df.actor_login == Core_contributor) & (Main_df.repo_name == current_repo)&(Main_df.created_at.between(start_date,end_date))&(Main_df.type=="ForkEvent")].id.count()

                if Fork_count_current_repo != 0:
                    decider=True
                else:
                    decider=False
                return decider


            # checking every cc in the main_repos from core_contributors_repos if he has forked current repo in the active period
            def subsequent_work_fun(current_repo,core_contributors_repos):
                subsequent_work=[]
                for key,value in core_contributors_repos.items():
                    for repo in value:
                        decider = reference_repos_bool(key,current_repo, repo, cc_df_other)
                        if decider==True:
                            subsequent_work.append(repo)
                        else:
                            continue
                subsequent_work=list(set(subsequent_work))
                return subsequent_work


            #calling the function to get the subsequent work
            subsequent_work=subsequent_work_fun(focal_repository,core_contributors_repos)
            #print("The susequent work citing focal repository are {}".format(subsequent_work))

            # getting the records containing all the subsequent work
            if len(subsequent_work)==0:
                D="NaN"
                Parameters=[D,len(contributors),len(core_contributors_main)]
                return Parameters
            if len(subsequent_work)==1:
                subsequent_work_t=(subsequent_work[0])
                query = 'SELECT * FROM gitdatam WHERE repo_name =%s'
                sub_df = pd.read_sql(query, con=db, params=[subsequent_work_t])
            else:
                subsequent_work_t = tuple(subsequent_work)
                query = "SELECT * FROM gitdatam WHERE repo_name IN {}".format(subsequent_work_t)
                sub_df = pd.read_sql(query, con=db)
            sub_df.head()


            # Function to get the contributors when repo is specified
            # pass the dataframe into the fuction containing all the types and repositories
            def contributors_repo_fun(df,repos_name):
                #creating a new data frame to store the subseted data(data pertaining a single repository and reponame
                Main_df=df.loc[(df['repo_name']==repos_name)].copy()
                Contributors=[]
                Contributors=Main_df.loc[(Main_df['type']=="PullRequestEvent")&(Main_df['payload_pull_request_merged']=="TRUE")].actor_login.tolist()
                unique_contributors= set(Contributors)
                return unique_contributors


            from collections import defaultdict
            cc_sub=[]
            repo_core_contributor = defaultdict(list)
            for repo in subsequent_work:
                unique_contributors_temp=contributors_repo_fun(sub_df,repo)
                core_contributors_temp=core_contributors_fun(unique_contributors_temp,sub_df)
                if core_contributors_temp!= []:
                    repo_core_contributor[repo]=core_contributors_temp

            for key,value in repo_core_contributor.items():
                #print(key)
                #print(value)
                cc_sub.extend(value)

            # printing all the core-contributors from repo_core_contributor
            #print(list(set(cc_sub)))

            # Getting the data frame containing the above cc's 
            #import pdb; pdb.set_trace()
            if len(cc_sub)==0 or len(reference_repos_list)==0:
                D="NaN"
                D="NaN"
                Parameters=[D,len(contributors),len(core_contributors_main)]
                return Parameters

            if len(cc_sub)==1 and len(reference_repos_list)==1:
                cc_sub_t = tuple(cc_sub)+("",)
                reference_repos_list_t = tuple(reference_repos_list)+("",)

            elif len(cc_sub)>1 and len(reference_repos_list)==1:
                cc_sub_t = tuple(cc_sub)
                reference_repos_list_t = tuple(reference_repos_list)+("",)
            elif len(cc_sub)==1 and len(reference_repos_list)>1:
                cc_sub_t = tuple(cc_sub)+("",)
                reference_repos_list_t = tuple(reference_repos_list)
            else:
                cc_sub_t = tuple(cc_sub)
                reference_repos_list_t = tuple(reference_repos_list)

            query = "SELECT * FROM gitdatam WHERE actor_login IN {} OR repo_name IN {}".format(cc_sub_t,reference_repos_list_t)
            cc_sub_df = pd.read_sql(query, con=db)
            cc_sub_df.head()

            #  Check if any of these core-contributors have forked any of the reference repositories in their active period.
            Subsequent_work_both = []
            for key, value in repo_core_contributor.items():
                for cc in value:
                    if reference_repos_list != []:
                        for ref_repo in reference_repos_list:
                            decider = reference_repos_bool(cc, ref_repo, key, cc_sub_df)
                            if decider == True:
                                Subsequent_work_both.append(key)
            Subsequent_work_both = list(set(Subsequent_work_both))

            print("The Subsequent work citing both the references and the focal repository are {}".format(Subsequent_work_both))
            subsequent_work_only=set(subsequent_work)-set(Subsequent_work_both)
            subsequent_work_only=list(subsequent_work_only)

            #print("The Subsequent work citing only the focal repository are {}".format(subsequent_work_only))


            ni= len(set(subsequent_work_only))
            nj=len(set(Subsequent_work_both))

            if (ni-nj)==0:
                print("The disruption value for {} is {}".format(focal_repository,(ni-nj)))
                D=0
                Parameters=[D,len(contributors),len(core_contributors_main)]
                return Parameters
            else:
                if len(reference_repos_list)==1:
                    reference_repos_list_t = tuple(reference_repos_list)+("",)
                else:
                    reference_repos_list_t = tuple(reference_repos_list)


                query = "SELECT DISTINCT * FROM gitdatam WHERE type= 'ForkEvent' AND repo_name IN {}".format(reference_repos_list_t)
                fork_df = pd.read_sql(query, con=db)
                fork_df.head()
                #Getting only the actors who forked the reference repositories
                actors= fork_df.actor_login.tolist()
                actors=list(set(actors))
                #print("The actors who forked reference repository are:{}".format(actors))
                print("The length of the actors are{}".format(len(actors)))
                # Getting the actors data into the dataframe
                #actors=["spatchcock","bensie"]
                if len(actors)==0:
                    D="NaN"
                    Parameters=[D,len(contributors),len(core_contributors_main)]
                    return Parameters
                if len(actors)==1:
                    actors_t = tuple(actors)+("",)
                else:
                    actors_t = tuple(actors)
                query = "SELECT DISTINCT * FROM gitdatam WHERE actor_login IN {}".format(actors_t)
                actors_df = pd.read_sql(query, con=db)
                actors_df.head()
                # step 3: Get all repositories to which these actors are contributors
                # function to get the contributors_repos dict, pass the dataframe and the actors
                #def contributors_actor_fun(df,actors):
                #import pdb; pdb.set_trace()
            #         from collections import defaultdict
            #         contributors_repos = defaultdict(list)
            #         for actor in actors:
            #             repo_list=df.loc[(df['type']=="PullRequestEvent")&(df['payload_pull_request_merged']=="TRUE")&(df['actor_login']==actor)].repo_name.tolist()
            #             repo_list=list(set(repo_list))
            #             if repo_list!=[]:
            #                 contributors_repos[actor]=repo_list
            #         return contributors_repos
                def contributors_actor_fun(df,actor):
                    #import pdb; pdb.set_trace()
                    from collections import defaultdict
                    contributors_repos = defaultdict(list)
                    repo_list=df.loc[(df['type']=="PullRequestEvent")&(df['payload_pull_request_merged']=="TRUE")&(df['actor_login']==actor)].repo_name.tolist()
                    repo_list=list(set(repo_list))
                    if repo_list!=[]:
                        contributors_repos[actor]=repo_list
                    #print(len(contributors_repos))
                    return contributors_repos


                # calling the function to get the contributors_repos dict based on the actors
                contributors_repos_last= pd.Series(actors).swifter.apply(lambda x:contributors_actor_fun(actors_df,x))
                #contributors_repos_last=contributors_actor_fun(actors_df,actors)
                contributors_repos_last=dict(contributors_repos_last)
                #print(contributors_repos_last
                repo_only_list=[]
                for key,value in contributors_repos_last.items():
                    repo_only_list.extend(value)
                repo_only_list=list(set(repo_only_list))
                #print(repo_only_list)
                #Getting all the repos for contributed by the contributors
                if len(repo_only_list)==0:
                    D="NaN"
                    Parameters=[D,len(contributors),len(core_contributors_main)]
                    return Parameters
                if len(repo_only_list)==1:
                    repo_only_list_t = tuple(repo_only_list)+("",)
                else:
                    repo_only_list_t = tuple(repo_only_list)
                query = "SELECT * FROM gitdatam WHERE repo_name IN {}".format(repo_only_list_t)
                repo_all_last = pd.read_sql(query, con=db)
                repo_all_last.head()
                # step 4: get all the repositories to which these contributors are core-contributors
                # Find out the core-contributors in the contributors_repos dictionary
                core_contributors_repos_last = core_contributors_repo_dict_fun(contributors_repos_last,repo_all_last)
                #print(core_contributors_repos_last)


                #step 5: Getting the repositories that the core contributors in core_contributors_repos has forked reference repos
                # while they were active on their repositories
                from collections import defaultdict
                reference_subsequent = defaultdict(list)
                subsequent_only_reference=[]
                for repo in reference_repos_list:
                    temp_repos=subsequent_work_fun(repo,core_contributors_repos)
                    reference_subsequent[repo]=temp_repos
                    subsequent_only_reference.extend(temp_repos)
                subsequent_only_reference=list(set(subsequent_only_reference))
                #print("The repositories citing only the refernces are {}".format(subsequent_only_reference))


                ni= len(set(subsequent_work_only))
                nj=len(set(Subsequent_work_both))
                nk=len(set(subsequent_only_reference))

                print("ni={}".format(ni))

                print("nj={}".format(nj))

                print("nk={}".format(nk))

                D=(ni-nj)/(ni+nj+nk)

                print("The disruption value for {} is {}".format(focal_repository,D))
                ABC=[D,len(contributors),len(core_contributors_main)]
                return ABC

import MySQLdb
import pandas as pd
import swifter
import xlsxwriter
# reading excel sheet
excel_df= pd.read_csv("search1.csv")
excel_df['D']=''
excel_df['Contributors']=''
excel_df['C_C']=''

# Open database connection
db = MySQLdb.connect("localhost","root","Root@123","github" )

name_list=list(excel_df.Repositories)
i=-1
for i in range(1,len(name_list)):
    focal_repository= name_list[i]
    i=i+1
    Parameter_list=get_disruption_value(focal_repository)
    excel_df.loc[i, 'D'] = Parameter_list[0]
    excel_df.loc[i, 'Contributors'] = Parameter_list[1]
    excel_df.loc[i, 'C_C'] = Parameter_list[2]
#     excel_df.to_excel("searchrepo.csv")
    
    print("{},{}: {}".format(i,focal_repository,Parameter_list[0]))

excel_df.to_csv("search1_values.csv")

The focal repository is projectblacklight/blacklight
The number of contributors are 4
The number of core contributors are 1
The Subsequent work citing both the references and the focal repository are ['projecthydra/solrizer', 'projecthydra/hydra-head', 'projecthydra/jettywrapper']
The length of the actors are24206
